## EDA del dataset de los viajes de HVFHV durante el mes de septiembre de 2024

### Paso 0: Importar librerias necesarias para el EDA y cargar el dataset que será analizado.

In [ ]:
#import pandas as pd
import numpy as numpy
#import matplotlib.pyplot as plt
#import seaborn as sns
import warnings
import pyarrow.parquet as pq
warnings.simplefilter("ignore")

* Pandas y NumPy para el manejo de la data.
* Matplotlib y seaborn para la visualizacion de los gráficos pertinentes.
* Warnings para legibilidad del código y visualizaciones.

In [14]:
trips = pq.read_table('C:/Users/NoxiePC/Desktop/henry/fhvhv_tripdata_2024-09.parquet')
trips = trips.to_pandas()

trips.head()

,hvfhs_license_num,dispatching_base_num,originating_base_num,request_datetime,on_scene_datetime,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,trip_miles,...,sales_tax,congestion_surcharge,airport_fee,tips,driver_pay,shared_request_flag,shared_match_flag,access_a_ride_flag,wav_request_flag,wav_match_flag
0,HV0003,B03404,B03404,2024-09-01 00:00:42,2024-09-01 00:04:11,2024-09-01 00:04:54,2024-09-01 00:12:03,263,75,1.64,...,0.75,0.75,0.0,0.00,6.40,Y,N,N,N,N
1,HV0003,B03404,B03404,2024-09-01 00:26:16,2024-09-01 00:27:03,2024-09-01 00:27:49,2024-09-01 00:48:40,263,249,5.66,...,4.04,2.75,0.0,0.00,29.99,N,N,N,N,N
2,HV0003,B03404,B03404,2024-09-01 00:07:08,2024-09-01 00:07:22,2024-09-01 00:08:57,2024-09-01 00:18:34,161,170,1.48,...,1.05,2.75,0.0,0.00,7.62,N,N,N,N,N
3,HV0003,B03404,B03404,2024-09-01 00:19:33,2024-09-01 00:21:14,2024-09-01 00:21:48,2024-09-01 00:46:17,233,220,11.52,...,2.96,2.75,0.0,0.00,29.94,N,N,N,N,N
4,HV0003,B03404,B03404,2024-09-01 00:45:51,2024-09-01 00:49:34,2024-09-01 00:49:34,2024-09-01 01:06:46,220,3,6.29,...,2.53,0.00,0.0,6.38,18.58,N,N,N,N,N


In [15]:
trips.shape

(19209788, 24)

El dataset contiene 19209788 filas, que representan los viajes realizados durante el mes de septiembre de 2024, y 24 columnas

In [16]:
trips.columns.tolist()

['hvfhs_license_num',
 'dispatching_base_num',
 'originating_base_num',
 'request_datetime',
 'on_scene_datetime',
 'pickup_datetime',
 'dropoff_datetime',
 'PULocationID',
 'DOLocationID',
 'trip_miles',
 'trip_time',
 'base_passenger_fare',
 'tolls',
 'bcf',
 'sales_tax',
 'congestion_surcharge',
 'airport_fee',
 'tips',
 'driver_pay',
 'shared_request_flag',
 'shared_match_flag',
 'access_a_ride_flag',
 'wav_request_flag',
 'wav_match_flag']

En lineas generales la información que contiene el dataset es la siguiente

* *Dispatching_base_num:* Número de licencia de la base de la Comisión de Taxis y Limusinas (TLC) que despachó el viaje. Esto identifica la base o empresa desde la cual se coordinó el viaje (por ejemplo, Uber, Lyft, etc.).
* *pickup_datetime:* Fecha y hora en que el pasajero fue recogido para el viaje.
* *DropOff_datetime:* Fecha y hora en que el pasajero fue dejado en su destino.
* *PULocationID:* Identificador del área de taxi TLC donde comenzó el viaje. Estas zonas están definidas por la TLC en la ciudad.
* *DOLocationID:* Identificador del área de taxi TLC donde terminó el viaje.
* *originating_base_num:* Número de la base que recibió originalmente la solicitud de viaje. Puede ser diferente de la base que despachó el viaje.
* *request_datetime:* Fecha y hora en que el pasajero solicitó ser recogido.
* *on_scene_datetime:* Fecha y hora en que el conductor llegó al lugar de recogida. Este dato aplica solo para vehículos accesibles (por ejemplo, para personas con movilidad reducida).
* *trip_miles:* Distancia total del viaje en millas.
* *trip_time:* Duración total del viaje en segundos.
* *base_passenger_fare:* Tarifa base pagada por el pasajero antes de incluir peajes, propinas, impuestos y otros cargos.
* *tolls:* Monto total de los peajes pagados durante el viaje.
* *bcf (Black Car Fund):* Monto recaudado en el viaje para el Black Car Fund, un fondo de beneficios para conductores en Nueva York.
* *sales_tax:* Monto recaudado en concepto de impuesto sobre las ventas del estado de Nueva York (NYS).
* *congestion_surcharge:* Monto recaudado por el recargo de congestión del estado de Nueva York.
* *airport_fee:* Tarifa fija de $2.50 para viajes que incluyen recogida o entrega en los aeropuertos LaGuardia, Newark o John F. Kennedy.
* *tips:* Total de propinas recibidas del pasajero.
* *driver_pay:* Pago neto recibido por el conductor (excluyendo peajes y propinas), después de comisiones, recargos e impuestos.
* *shared_request_flag:* Indica si el pasajero aceptó compartir un viaje o una tarifa combinada con otros pasajeros.  
  * **Y:** Sí, aceptó compartir el viaje.  
  * **N:** No, no aceptó compartir el viaje.
* *shared_match_flag:* Indica si el pasajero efectivamente compartió el vehículo con otra persona que reservó el viaje por separado.  
  * **Y:** Sí, compartió el vehículo.  
  * **N:** No, no compartió el vehículo.
* *access_a_ride_flag:* Indica si el viaje fue administrado en nombre de la Autoridad de Transporte Metropolitano (MTA).  
  * **Y:** Sí, el viaje fue administrado por la MTA.  
  * **N:** No, no fue administrado por la MTA.
* *wav_request_flag:* Indica si el pasajero solicitó un vehículo accesible para sillas de ruedas (WAV).  
  * **Y:** Sí, solicitó un WAV.  
  * **N:** No, no solicitó un WAV.
* *wav_match_flag:* Indica si el viaje se realizó en un vehículo accesible para sillas de ruedas (WAV).  
  * **Y:** Sí, el viaje fue en un WAV.  
  * **N:** No, el viaje no fue en un WAV.


In [17]:
trips.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19209788 entries, 0 to 19209787
Data columns (total 24 columns):
 #   Column                Dtype         
---  ------                -----         
 0   hvfhs_license_num     object        
 1   dispatching_base_num  object        
 2   originating_base_num  object        
 3   request_datetime      datetime64[us]
 4   on_scene_datetime     datetime64[us]
 5   pickup_datetime       datetime64[us]
 6   dropoff_datetime      datetime64[us]
 7   PULocationID          int32         
 8   DOLocationID          int32         
 9   trip_miles            float64       
 10  trip_time             int64         
 11  base_passenger_fare   float64       
 12  tolls                 float64       
 13  bcf                   float64       
 14  sales_tax             float64       
 15  congestion_surcharge  float64       
 16  airport_fee           float64       
 17  tips                  float64       
 18  driver_pay            float64       
 19

In [18]:
# Se ordena la data de acuerdo a la hora y fecha que comienza el viaje
trips.sort_values(by=['pickup_datetime'])

,hvfhs_license_num,dispatching_base_num,originating_base_num,request_datetime,on_scene_datetime,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,trip_miles,...,sales_tax,congestion_surcharge,airport_fee,tips,driver_pay,shared_request_flag,shared_match_flag,access_a_ride_flag,wav_request_flag,wav_match_flag
20056,HV0003,B03404,B03404,2024-08-31 23:56:32,2024-08-31 23:59:13,2024-09-01 00:00:00,2024-09-01 00:17:01,114,66,3.050,...,2.17,2.75,0.0,0.00,14.07,N,N,N,N,N
28937,HV0003,B03404,B03404,2024-08-31 23:59:12,2024-08-31 23:59:40,2024-09-01 00:00:00,2024-09-01 00:39:54,170,37,8.350,...,3.26,2.75,0.0,0.00,34.62,N,N,N,N,N
13421,HV0005,B03406,None,2024-09-01 00:00:01,NaT,2024-09-01 00:00:00,2024-09-01 00:16:36,101,218,10.410,...,3.18,0.00,0.0,0.00,23.92,N,N,N,N,N
22462,HV0003,B03404,B03404,2024-08-31 23:58:58,2024-08-31 23:59:43,2024-09-01 00:00:00,2024-09-01 00:14:22,261,170,4.610,...,1.92,2.75,0.0,0.00,14.65,N,N,N,N,N
27318,HV0003,B03404,B03404,2024-08-31 23:56:13,2024-09-01 00:00:00,2024-09-01 00:00:00,2024-09-01 00:07:05,225,37,1.050,...,0.79,0.00,0.0,0.00,5.56,N,N,N,N,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19197899,HV0003,B03404,B03404,2024-09-30 23:58:23,2024-09-30 23:58:56,2024-09-30 23:59:59,2024-10-01 00:04:53,164,170,0.750,...,1.59,2.75,0.0,0.00,10.62,N,N,N,N,N
19197363,HV0003,B03404,B03404,2024-09-30 23:57:13,2024-09-30 23:59:29,2024-09-30 23:59:59,2024-10-01 00:12:24,197,216,2.520,...,1.11,0.00,0.0,3.00,10.67,N,N,N,N,N
19202126,HV0005,B03406,None,2024-09-30 23:58:16,NaT,2024-09-30 23:59:59,2024-10-01 00:23:38,163,116,6.227,...,2.28,2.75,0.0,0.00,25.41,N,N,N,N,N
19192658,HV0005,B03406,None,2024-09-30 23:57:17,NaT,2024-09-30 23:59:59,2024-10-01 00:07:06,50,143,0.861,...,0.66,2.75,0.0,0.00,5.67,N,N,N,N,Y


In [ ]:
# Se verifica si hay valores de fecha que no correspondan a septiembre de 2024
trips[(trips['pickup_datetime'].dt.year != 2024) | (trips['pickup_datetime'].dt.month != 9)]

,hvfhs_license_num,dispatching_base_num,originating_base_num,request_datetime,on_scene_datetime,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,trip_miles,...,sales_tax,congestion_surcharge,airport_fee,tips,driver_pay,shared_request_flag,shared_match_flag,access_a_ride_flag,wav_request_flag,wav_match_flag


No se encontraron valores de fecha fuera de septiembre de 2024, que es el rango tiempo en el que se tomaron los datos

In [26]:
# Se observan valores nulos
trips.isnull().sum()

hvfhs_license_num             0
dispatching_base_num          0
originating_base_num    4571894
request_datetime              0
on_scene_datetime       4571854
pickup_datetime               0
dropoff_datetime              0
PULocationID                  0
DOLocationID                  0
trip_miles                    0
trip_time                     0
base_passenger_fare           0
tolls                         0
bcf                           0
sales_tax                     0
congestion_surcharge          0
airport_fee                   0
tips                          0
driver_pay                    0
shared_request_flag           0
shared_match_flag             0
access_a_ride_flag            0
wav_request_flag              0
wav_match_flag                0
dtype: int64

Se observa que las columnas **originating_base_num** y **on_scene_datetime** tienen un 2,4% de valores nulos

Estos faltantes pueden deberse a errores de registro

In [27]:
# Se observan valores duplicados 
trips.duplicated().sum()

np.int64(0)

No se encontraron valores duplicados en este dataset

Se procede a evaluar algunos valores por columna.

Para ciertas columnas, un valor cero o negativo no tiene sentido...

In [28]:
print(f'Viajes con distancia 0: {trips[trips['trip_miles'] == 0].shape[0]}')
print(f'Viajes con distancia "negativa": {trips[trips['trip_miles'] < 0].shape[0]}')

Viajes con distancia 0: 2991
Viajes con distancia "negativa": 0


No hay valores de distancia negativos registrados

Hay 2991 (0.02%) de valores de distancia iguales a cero. <br>Esto puede deberse a un error al activar el contador de millas del taxi

In [41]:
# Se buscan si se registraron inicios o finales de viaje fuera de las locaciones existentes
print(trips[trips['PULocationID'] <= 0].shape)
print(trips[trips['DOLocationID'] <= 0].shape)

print(trips[trips['PULocationID'] >= 266].shape)
print(trips[trips['DOLocationID'] >= 266].shape)

(0, 24)
(0, 24)
(0, 24)
(0, 24)


In [42]:
print(f'Viajes con tarifa 0: {trips[trips['base_passenger_fare'] == 0].shape[0]}')
print(f'Viajes con tarifa "negativa": {trips[trips['base_passenger_fare'] < 0].shape[0]}')

Viajes con tarifa 0: 1188
Viajes con tarifa "negativa": 477


In [43]:
# 
df_filtered = trips.dropna(subset=["request_datetime", "on_scene_datetime"])
df_filtered["wait_time_minutes"] = (df_filtered["on_scene_datetime"] - df_filtered["request_datetime"]).dt.total_seconds() / 60

average_wait_time = df_filtered["wait_time_minutes"].mean()

print(f"El tiempo de espera promedio es de {average_wait_time:.2f} minutos.")


El tiempo de espera promedio es de 3.47 minutos.
